In [107]:
import pickle
import os
from collections import defaultdict, Counter
from math import isnan

import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import log_loss
from scipy.stats import kendalltau, spearmanr

## 1

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.


In [3]:
dir_path = 'chgk'
data = {}
for file in os.listdir(dir_path):
    with open(f'{dir_path}/{file}', 'rb') as f:
        data[file.split('.')[0]] = pickle.load(f)
data.keys()

dict_keys(['results', 'players', 'tournaments'])

In [4]:
def get_usefull_data(data, asseptable_years):
    """
    Выбираем игры проходившие в assebtable_years, где есть маска
    и инофрмация о членах команды хотябы по одной команде
    """
    results_selection = {}
    info_check = lambda team: team.get('mask', None) and team['teamMembers'] != []

    for k, v in data['results'].items():
        year = data['tournaments'][k]['dateStart'][:4]
        if year in asseptable_years:
            teams_with_info = list(filter(info_check, v))
            if len(teams_with_info) != 0:
                results_selection[k] = (year, teams_with_info)
    
    return results_selection

In [5]:
asseptable_years = {'2019', '2020'}

In [6]:
results = get_usefull_data(data, asseptable_years)

In [7]:
def data_to_pandas(results):
    """Функция для перевода данных в pandas"""
    data = defaultdict(list)
    
    for k, v in results.items():
        year, teams = v
        for turn in teams:
            data['year'].append(year)
            data['tournament'].append(k)
            data['teamid'].append(turn["team"]["id"])
            data['mask'].append(turn["mask"])
            data['position'].append(turn["position"])
            data['members'].append([iterr["player"]["id"] for iterr in turn['teamMembers']])
        
    return pd.DataFrame(data)

In [33]:
df = data_to_pandas(results)
df

,year,tournament,teamid,mask,position,members
0,2019,4772,45556,111111111011111110111111111100010010,1.0,"[6212, 18332, 18036, 22799, 15456, 26089]"
1,2019,4772,1030,111111111011110100101111011001011010,5.5,"[1585, 40840, 1584, 10998, 16206]"
2,2019,4772,4252,111111111011110101101111001011110000,5.5,"[23513, 18168, 21060, 35850, 31332, 10187]"
3,2019,4772,5444,101111101111111110001101011001111010,5.5,"[36742, 28939, 54289, 15381, 27375]"
4,2019,4772,40931,111111101011111101000111001001111110,5.5,"[28689, 17720, 30597, 12400, 26988, 69476]"
...,...,...,...,...,...,...
108756,2020,6456,55612,100110001100111011011110100100010101010,3.0,"[138479, 80384, 133207, 95145, 121594, 133209]"
108757,2020,6456,68457,100110101100001111110110100100010001010,4.5,[64129]
108758,2020,6456,43261,000001111100111001010110000100111001011,4.5,"[111255, 166584, 135103, 103290, 103293]"
108759,2020,6456,69918,100101101100100100010110100100010001010,6.0,"[96552, 192899, 192900, 123238, 129706, 192901]"


## 2

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков.
Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [39]:
def get_tuple_score(mask):
    tournament = mask[0]
    mask_up = []
    true_ans = 0
    max_win_strike = 0
    last_win_strike = 0
    for idx, i in enumerate(mask[1:]):
        mask_up.append({
            'name' : f'{tournament}_{idx}', 
            'result' : i,
            'true_ans_before' : true_ans,
            'question_number' : idx,
            'last_win_strike' : last_win_strike,
            'max_win_strike' : max_win_strike
        })
        int_i = int(i) if i in {'0', '1'} else 0
        last_win_strike = (last_win_strike + int_i) * int_i
        max_win_strike = max([max_win_strike, last_win_strike])
        true_ans += int_i
    
    return mask_up

In [34]:
df['team_score'] = df['mask'].apply(lambda x: sum(list(map(lambda x: int(x) if x in {'0', '1'} else 0, x))))
df['mask'] = df['tournament'].apply(lambda x:[str(x)]) + df['mask'].apply(lambda x: list(map(str, x)))
df

,year,tournament,teamid,mask,position,members,team_score
0,2019,4772,45556,"[4772, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",1.0,"[6212, 18332, 18036, 22799, 15456, 26089]",28
1,2019,4772,1030,"[4772, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",5.5,"[1585, 40840, 1584, 10998, 16206]",25
2,2019,4772,4252,"[4772, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",5.5,"[23513, 18168, 21060, 35850, 31332, 10187]",25
3,2019,4772,5444,"[4772, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...",5.5,"[36742, 28939, 54289, 15381, 27375]",25
4,2019,4772,40931,"[4772, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, ...",5.5,"[28689, 17720, 30597, 12400, 26988, 69476]",25
...,...,...,...,...,...,...,...
108756,2020,6456,55612,"[6456, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, ...",3.0,"[138479, 80384, 133207, 95145, 121594, 133209]",20
108757,2020,6456,68457,"[6456, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, ...",4.5,[64129],19
108758,2020,6456,43261,"[6456, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, ...",4.5,"[111255, 166584, 135103, 103290, 103293]",19
108759,2020,6456,69918,"[6456, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, ...",6.0,"[96552, 192899, 192900, 123238, 129706, 192901]",16


In [40]:
df['mask'] = df['mask'].apply(get_tuple_score)
df

,year,tournament,teamid,mask,position,members,team_score
0,2019,4772,45556,"[{'name': '4772_0', 'result': '1', 'true_ans_b...",1.0,"[6212, 18332, 18036, 22799, 15456, 26089]",28
1,2019,4772,1030,"[{'name': '4772_0', 'result': '1', 'true_ans_b...",5.5,"[1585, 40840, 1584, 10998, 16206]",25
2,2019,4772,4252,"[{'name': '4772_0', 'result': '1', 'true_ans_b...",5.5,"[23513, 18168, 21060, 35850, 31332, 10187]",25
3,2019,4772,5444,"[{'name': '4772_0', 'result': '1', 'true_ans_b...",5.5,"[36742, 28939, 54289, 15381, 27375]",25
4,2019,4772,40931,"[{'name': '4772_0', 'result': '1', 'true_ans_b...",5.5,"[28689, 17720, 30597, 12400, 26988, 69476]",25
...,...,...,...,...,...,...,...
108756,2020,6456,55612,"[{'name': '6456_0', 'result': '1', 'true_ans_b...",3.0,"[138479, 80384, 133207, 95145, 121594, 133209]",20
108757,2020,6456,68457,"[{'name': '6456_0', 'result': '1', 'true_ans_b...",4.5,[64129],19
108758,2020,6456,43261,"[{'name': '6456_0', 'result': '0', 'true_ans_b...",4.5,"[111255, 166584, 135103, 103290, 103293]",19
108759,2020,6456,69918,"[{'name': '6456_0', 'result': '1', 'true_ans_b...",6.0,"[96552, 192899, 192900, 123238, 129706, 192901]",16


In [41]:
df_players = df.explode('members').explode('mask')
df_players

,year,tournament,teamid,mask,position,members,team_score
0,2019,4772,45556,"{'name': '4772_0', 'result': '1', 'true_ans_be...",1.0,6212,28
0,2019,4772,45556,"{'name': '4772_1', 'result': '1', 'true_ans_be...",1.0,6212,28
0,2019,4772,45556,"{'name': '4772_2', 'result': '1', 'true_ans_be...",1.0,6212,28
0,2019,4772,45556,"{'name': '4772_3', 'result': '1', 'true_ans_be...",1.0,6212,28
0,2019,4772,45556,"{'name': '4772_4', 'result': '1', 'true_ans_be...",1.0,6212,28
...,...,...,...,...,...,...,...
108760,2020,6456,63129,"{'name': '6456_34', 'result': '0', 'true_ans_b...",7.0,224329,13
108760,2020,6456,63129,"{'name': '6456_35', 'result': '0', 'true_ans_b...",7.0,224329,13
108760,2020,6456,63129,"{'name': '6456_36', 'result': '0', 'true_ans_b...",7.0,224329,13
108760,2020,6456,63129,"{'name': '6456_37', 'result': '0', 'true_ans_b...",7.0,224329,13


In [42]:
new_cols = ['name', 
            'result',
            'true_ans_before',
            'question_number',
            'last_win_strike',
            'max_win_strike']

In [43]:
for col in new_cols:
    df_players[col] = df_players['mask'].apply(lambda x: x[col])
df_players

,year,tournament,teamid,mask,position,members,team_score,name,result,true_ans_before,question_number,last_win_strike,max_win_strike
0,2019,4772,45556,"{'name': '4772_0', 'result': '1', 'true_ans_be...",1.0,6212,28,4772_0,1,0,0,0,0
0,2019,4772,45556,"{'name': '4772_1', 'result': '1', 'true_ans_be...",1.0,6212,28,4772_1,1,1,1,1,1
0,2019,4772,45556,"{'name': '4772_2', 'result': '1', 'true_ans_be...",1.0,6212,28,4772_2,1,2,2,2,2
0,2019,4772,45556,"{'name': '4772_3', 'result': '1', 'true_ans_be...",1.0,6212,28,4772_3,1,3,3,3,3
0,2019,4772,45556,"{'name': '4772_4', 'result': '1', 'true_ans_be...",1.0,6212,28,4772_4,1,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108760,2020,6456,63129,"{'name': '6456_34', 'result': '0', 'true_ans_b...",7.0,224329,13,6456_34,0,12,34,1,2
108760,2020,6456,63129,"{'name': '6456_35', 'result': '0', 'true_ans_b...",7.0,224329,13,6456_35,0,12,35,0,2
108760,2020,6456,63129,"{'name': '6456_36', 'result': '0', 'true_ans_b...",7.0,224329,13,6456_36,0,12,36,0,2
108760,2020,6456,63129,"{'name': '6456_37', 'result': '0', 'true_ans_b...",7.0,224329,13,6456_37,0,12,37,0,2


In [45]:
df_players['name_hash'] = df_players['name'].apply(lambda x: hash(x))

In [46]:
features =  ['name_hash',
            'tournament',
            'teamid',
            'true_ans_before',
            'question_number',
            'last_win_strike',
            'max_win_strike']
target = 'result'

In [49]:
train, test = df_players.query('year == "2019"'), df_players.query('year == "2020"')
X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]
print(f'Train: {X_train.shape}\nTest: {X_test.shape}')

Train: (21014267, 7)
Test: (4483123, 7)


### Обучение модели

In [52]:
model = linear_model.LogisticRegression()
model.fit(X_train, y_train)

pred_train = model.predict_proba(X_train)
pred_test = model.predict_proba(X_test)
print(f"Train log_loss: {log_loss(y_train, pred_train)}\nTest log_loss:  {log_loss(y_test, pred_test)}")

Train log_loss: 1.3862687433387129
Test log_loss:  1.386110527325844


## 3

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Идея:
- Кажется, что команда отвечает настолько хорошо, наскольлко силен ее самый сильный игрок.
- Предлагаю ранжировать команды по среднему баллу за все игры у сильнейшего игрока
- Но если игрок попал в команду с сильным игроком и выиграл, при этом учавствовал один раз, он будет считаться сильным игроком. Поэтому предлагается оттдавать приоритет игрокам с большим количеством игр. будем умножать средний балл на мультипликатор = количество игр игрока / количество игр всего
- Но и остальные игроки могу внести свой вклад, поэтому будем их считать рейтингом вполовину меньше(рейтинг игрока 1  + рейтинг игрока 2 * 0.5 + рейтинг игрока 3 * 0.25 и тд)
- По итоговому значению будем сортировать игроков

### Расчет индивидуального рейтинга игрока

In [55]:
df.head()

,year,tournament,teamid,mask,position,members,team_score
0,2019,4772,45556,"[{'name': '4772_0', 'result': '1', 'true_ans_b...",1.0,"[6212, 18332, 18036, 22799, 15456, 26089]",28
1,2019,4772,1030,"[{'name': '4772_0', 'result': '1', 'true_ans_b...",5.5,"[1585, 40840, 1584, 10998, 16206]",25
2,2019,4772,4252,"[{'name': '4772_0', 'result': '1', 'true_ans_b...",5.5,"[23513, 18168, 21060, 35850, 31332, 10187]",25
3,2019,4772,5444,"[{'name': '4772_0', 'result': '1', 'true_ans_b...",5.5,"[36742, 28939, 54289, 15381, 27375]",25
4,2019,4772,40931,"[{'name': '4772_0', 'result': '1', 'true_ans_b...",5.5,"[28689, 17720, 30597, 12400, 26988, 69476]",25


In [69]:
players = (
    df
    .query('year == "2019"')
    [['members', 'team_score']]
    .explode('members')
    .groupby('members')
    .agg(
        score = ('team_score', 'sum'),
        games = ('team_score', 'count')
    )
)
total_games = df.tournament.nunique()
players['raiting'] = players['score'] * players['games'] / total_games
player_dict = players['raiting'].to_dict()
player_dict

{15: 3.57311320754717,
 16: 0.6084905660377359,
 23: 0.02122641509433962,
 31: 12.037735849056604,
 35: 11.88679245283019,
 38: 1.7122641509433962,
 47: 4.35377358490566,
 59: 3.2523584905660377,
 65: 0.7924528301886793,
 79: 21.75,
 80: 11.556603773584905,
 82: 31.995283018867923,
 98: 0.13797169811320756,
 112: 0.16981132075471697,
 113: 0.30778301886792453,
 117: 80.66037735849056,
 119: 4.476415094339623,
 133: 13.702830188679245,
 136: 1.1226415094339623,
 144: 24.452830188679247,
 150: 2.806603773584906,
 153: 0.660377358490566,
 157: 0.024764150943396228,
 160: 0.02240566037735849,
 176: 91.875,
 178: 28.27240566037736,
 182: 31.482311320754718,
 196: 0.38207547169811323,
 223: 0.20047169811320756,
 230: 6.339622641509434,
 232: 28.49174528301887,
 233: 4.169811320754717,
 236: 0.125,
 261: 0.08018867924528301,
 263: 0.07311320754716981,
 278: 29.537735849056602,
 286: 0.020047169811320754,
 315: 3.231132075471698,
 322: 0.07311320754716981,
 323: 254.7311320754717,
 328: 74.136

### Расчет командного рейтинга

In [76]:
def team_score(team):
    score = []
    mul = 1
    total = 0
    for member in team:
        score.append(player_dict.get(member, 0))
    for member_score in sorted(score, reverse=True):
        total += mul * member_score
        mul *= 0.5
        
    return total

In [89]:
df['team_raiting'] = df['members'].apply(team_score)
df_to_score = df[['year', 'tournament', 'position', 'team_raiting']].sort_values(['tournament', 'position'])

In [90]:
train = df_to_score.query('year == "2019"')
test = df_to_score.query('year == "2020"')

In [93]:
train.query('tournament==4772')

,year,tournament,position,team_raiting
0,2019,4772,1.0,495.457953
1,2019,4772,5.5,165.739239
2,2019,4772,5.5,99.530218
3,2019,4772,5.5,93.865566
4,2019,4772,5.5,299.050044
...,...,...,...,...
226,2019,4772,228.5,71.704599
227,2019,4772,228.5,45.413915
228,2019,4772,228.5,16.724351
229,2019,4772,228.5,1.161225


### Расчет метрик

In [108]:
def calc_metrics(df):
    spearman = []
    kendal = []
    for tour in df.tournament.unique():
        true = df[df.tournament == tour].position
        pred = df[df.tournament == tour].team_raiting
        k = kendalltau(pred, true).correlation
        s = spearmanr(pred, true).correlation
        kendal.append(0 if isnan(k) else k)
        spearman.append(0 if isnan(s) else s)
        
    print(f'Mean Spearman corr: {np.mean(spearman)}\nMean Kendal corr: {np.mean(kendal)}')

In [111]:
print('Train')
calc_metrics(train)
print('Test')
calc_metrics(test)

Train
Mean Spearman corr: -0.5988371695993082
Mean Kendal corr: -0.45148554585513667
Test
Mean Spearman corr: -0.6152717157037964
Mean Kendal corr: -0.46263622364349916
